#### Titanic Data Classifier

### Initilise Mlflow tracking

In [66]:
## enable autologging
experiment_name = "DecisionTreeClassifier"
mlflow.set_experiment(experiment_name)
mlflow.sklearn.autolog(experiment_name)

2024/11/28 07:59:28 INFO mlflow.tracking.fluent: Experiment with name 'DecisionTreeClassifier' does not exist. Creating a new experiment.


In [67]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import mlflow
import mlflow.sklearn

In [68]:
df = pd.read_csv("../datasets/titanic.csv")

In [69]:
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


### Objective
Create a ML Classifier to predict wheather a person will survive the titanic accident or not

#### Pointers/findings  about dataset

##### *`survived`* is the target variable

*`Embarked`* starting point of the person

*`Note: In Machine learning, we makes sure to not use correlated features`*



In [70]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [71]:
df = df.fillna(0) # not good practice

In [72]:
df.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,0,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,0,S


In [73]:
# category encoding - here all the males converted to 1 and female as 0
df["gender_enc"] = df["Sex"].astype("category").cat.codes

In [74]:
df["embark_enc"] = df["Embarked"].astype("category").cat.codes

In [75]:
X = df[["Pclass","Age","gender_enc","embark_enc","Fare","SibSp","Parch"]]
Y = df["Survived"]

#### Test-Train split the data

In [76]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

### classifier

In [83]:
## Step - 1 initialise the model class

model = DecisionTreeClassifier(criterion="entropy",max_depth=7)

In [84]:
## Step - 2 train the model over training data
model.fit(X_train, y_train)

2024/11/28 08:00:45 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'd6fad76bf88d42d386e5bf6a3413b350', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow
2024/11/28 08:00:45 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/narkhann/miniconda3/envs/mlops/lib/python3.8/site-packages/mlflow/types/utils.py:407: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers Wi

DecisionTreeClassifier(criterion='entropy', max_depth=7)

#### How to find which parameter to tune or Hypertuning

*`DecisionTreeClassifier?`* - Name of the model followed by ?

In [79]:
DecisionTreeClassifier?

Init signature:
DecisionTreeClassifier(
    *,
    criterion='gini',
    splitter='best',
    max_depth=None,
    min_samples_split=2,
    min_samples_leaf=1,
    min_weight_fraction_leaf=0.0,
    max_features=None,
    random_state=None,
    max_leaf_nodes=None,
    min_impurity_decrease=0.0,
    class_weight=None,
    ccp_alpha=0.0,
)
Docstring:     
A decision tree classifier.

Read more in the :ref:`User Guide <tree>`.

Parameters
----------
criterion : {"gini", "entropy", "log_loss"}, default="gini"
    The function to measure the quality of a split. Supported criteria are
    "gini" for the Gini impurity and "log_loss" and "entropy" both for the
    Shannon information gain, see :ref:`tree_mathematical_formulation`.

splitter : {"best", "random"}, default="best"
    The strategy used to choose the split at each node. Supported
    strategies are "best" to choose the best split and "random" to choose
    the best random split.

max_depth : int, default=None
    The maximum depth o

In [85]:
## Step - 3 predict this over test set
y_pred = model.predict(X_test)

2024/11/28 08:00:53 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/narkhann/miniconda3/envs/mlops/lib/python3.8/site-packages/mlflow/types/utils.py:407: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."


### Model Evaluation

In [86]:
accuracy_score(y_test,y_pred)*100

78.35820895522389